# 🏛️ Arquitecturas Modernas de Datos: Lambda, Kappa, Delta y Data Mesh

Objetivo: comprender y contrastar arquitecturas de referencia (Lambda, Kappa, Delta) y patrones organizacionales (Data Mesh), con ejemplos de diseño y trade-offs.

- Duración: 120–150 min
- Dificultad: Alta
- Prerrequisitos: Mid completo, experiencia con batch y streaming

## 1. Arquitectura Lambda

- Batch layer: histórico completo, recalculable, alta latencia (Spark/Hadoop).
- Speed layer: streaming incremental, baja latencia (Kafka/Flink/Spark Streaming).
- Serving layer: merge de vistas batch + speed para consultas (Druid/Cassandra/BigQuery).
- Trade-offs: doble lógica (batch + stream), complejidad operacional, eventual consistency.

In [ ]:
lambda_diagram = '''
┌────────────┐
│   Data     │
│  Sources   │
└─────┬──────┘
      │
   ┌──┴───┐
   │Queue │ (Kafka)
   └──┬───┘
      │
 ┌────┴─────┐
 │  Batch   │  (Spark jobs nocturnos)
 │  Layer   │
 └────┬─────┘
      │
   Master
   Dataset
      │
   ┌──┴───┐
   │Serving│ (Druid/BigQuery)
   │ Layer │
   └──────┘
      ↑
   ┌──┴───┐
   │Speed │ (Flink/Spark Streaming)
   │Layer │
   └──────┘
'''
print(lambda_diagram)

## 2. Arquitectura Kappa

- Un solo pipeline streaming para todo (batch = replay del log).
- Simplifica operaciones y lógica unificada.
- Requiere log infinito (Kafka con retención larga) y capacidad de reprocessing.
- Trade-offs: coste de almacenamiento del log, complejidad del estado, menor optimización batch.

## 3. Arquitectura Delta (Lakehouse)

- Unifica batch y streaming sobre un único storage transaccional (Delta/Iceberg).
- Streaming escribe micro-batches transaccionales; batch lee histórico con time travel.
- Elimina duplicación de código y simplifica gobernanza (un catálogo, un linaje).
- Trade-offs: requiere adopción de formato y motor compatible (Spark/Trino).

## 4. Data Mesh: paradigma organizacional

- Descentralización de la propiedad de datos por dominio de negocio.
- Data products: APIs/tablas con SLOs, documentación, linaje y contratos.
- Plataforma self-service: herramientas comunes (CI/CD, catálogo, observabilidad).
- Gobernanza federada: políticas globales + autonomía local.
- Trade-offs: requiere madurez organizacional, infraestructura sólida, cambio cultural.

In [ ]:
mesh_principles = '''
1. Domain-oriented decentralization: equipos de dominio (ventas, logística, finanzas) son dueños de sus datos.
2. Data as a product: cada dataset tiene owner, SLO, versionado, documentación y calidad garantizada.
3. Self-serve platform: infraestructura común (orquestación, observabilidad, catálogo) sin silos.
4. Federated governance: políticas de seguridad, privacidad y calidad definidas centralmente, aplicadas localmente.
'''
print(mesh_principles)

## 5. Comparación y cuándo elegir cada una

In [ ]:
import pandas as pd
comp = pd.DataFrame([
  {'Arquitectura':'Lambda', 'Latencia':'Batch: horas, Speed: segundos', 'Complejidad':'Alta (doble lógica)', 'Casos':'Legacy con requerimientos mixtos'},
  {'Arquitectura':'Kappa', 'Latencia':'Baja (streaming)', 'Complejidad':'Media (un pipeline)', 'Casos':'Todo es streaming, reprocessing factible'},
  {'Arquitectura':'Delta', 'Latencia':'Baja-Media (micro-batch)', 'Complejidad':'Media (un storage)', 'Casos':'Lakehouse, unificación batch/stream'},
  {'Arquitectura':'Data Mesh', 'Latencia':'Variable (por dominio)', 'Complejidad':'Alta (organizacional)', 'Casos':'Grandes org con múltiples dominios'}
])
comp

## 6. Ejercicio de diseño

Diseña la arquitectura de datos para un e-commerce con:
- Dominio Ventas: transacciones en tiempo real (Kafka).
- Dominio Logística: batch nocturno (inventario/envíos).
- Dominio Analítica: dashboards con latencia < 5 min.
- Requisitos: auditabilidad, GDPR, linaje, costos optimizados.

Responde: ¿Lambda, Kappa, Delta o híbrida? ¿Data Mesh aplicable? Justifica con trade-offs.